In [47]:
import pandas as pd
import numpy as np
import os
import glob

In [48]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_columns', 115)

In [49]:
raw_data_path = "../data/raw"

# Get a list of all CSV files matching your pattern
csv_files = glob.glob(os.path.join(raw_data_path, "stats_player_reg_*.csv"))

dfs = []
for file in csv_files:
    print(f"Loading {file}")
    df = pd.read_csv(file)
    
    # Extract year from filename, e.g., stats_player_reg_2023.csv
    year = int(os.path.basename(file).split('_')[-1].split('.')[0])
    df['season'] = year
    
    dfs.append(df)

# Concatenate all seasons into one DataFrame
all_seasons = pd.concat(dfs, ignore_index=True)

# Filter only skill positions
skill_positions = ["QB", "RB", "WR", "TE"]
all_seasons = all_seasons[all_seasons['position'].isin(skill_positions)].reset_index(drop = True).copy()


print(f"Combined dataset shape: {all_seasons.shape}")
print(all_seasons['season'].value_counts())
print(all_seasons['position'].value_counts())

Loading ../data/raw/stats_player_reg_2017.csv
Loading ../data/raw/stats_player_reg_2016.csv
Loading ../data/raw/stats_player_reg_2015.csv
Loading ../data/raw/stats_player_reg_2022.csv
Loading ../data/raw/stats_player_reg_2023.csv
Loading ../data/raw/stats_player_reg_2021.csv
Loading ../data/raw/stats_player_reg_2020.csv
Loading ../data/raw/stats_player_reg_2018.csv
Loading ../data/raw/stats_player_reg_2024.csv
Loading ../data/raw/stats_player_reg_2019.csv
Combined dataset shape: (5815, 113)
season
2021    632
2022    608
2020    599
2024    588
2023    576
2018    576
2019    571
2016    557
2015    556
2017    552
Name: count, dtype: int64
position
WR    2283
RB    1504
TE    1262
QB     766
Name: count, dtype: int64


In [50]:
all_seasons.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/{form...,2017,REG,NE,16,385,581,4577,32,8,35,-201,6,2,5308,1910,230,140.614184,3.735653,2,0.862283,25,28,0,1,1,9,-12.203785,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.00000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,2,3,0,0,0,2,15,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,295.88,295.88
1,00-0020531,D.Brees,Drew Brees,QB,QB,https://static.www.nfl.com/image/private/{form...,2017,REG,NO,16,386,536,4334,23,8,20,-145,2,0,3417,2369,201,106.646449,4.681357,0,1.268364,33,12,2,2,0,10,-8.335352,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.00000,0.000000,0.000000,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,3,5,0,0,0,5,32,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,262.56,262.56
2,00-0021206,J.McCown,Josh McCown,QB,QB,https://static.www.nfl.com/image/private/{form...,2017,REG,NYJ,13,267,397,2926,18,9,39,-264,6,3,3036,1268,135,13.373195,5.534142,0,0.963768,37,124,5,4,1,17,6.621054,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.00000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,6,9,0,0,0,4,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,205.44,205.44
3,00-0021429,C.Palmer,Carson Palmer,QB,QB,https://static.www.nfl.com/image/private/{form...,2017,REG,ARI,7,164,267,1978,9,7,22,-150,0,0,2499,743,95,-4.198268,-0.034985,0,0.791517,14,12,0,2,0,6,0.243019,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.00000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,2,3,0,0,0,1,5,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,102.32,102.32
4,00-0021547,A.Gates,Antonio Gates,TE,TE,https://static.www.nfl.com/image/private/{form...,2017,REG,LAC,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,30,52,316,3,0,0,432,66,16,3.038502,0,0.731481,0.09075,0.090377,0.199389,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,49.60,79.60


In [51]:
qb_all = all_seasons[all_seasons['position'] == 'QB']

qb_filtered = qb_all[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 'completions', 'attempts', 'passing_yards',
             'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_fumbles', 'passing_first_downs', 'passing_epa', 
             'passing_cpoe', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs', 'rushing_epa',
             'fantasy_points_ppr'
             ]].copy()

qb_filtered.sort_values(by = 'fantasy_points_ppr', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_fumbles,passing_first_downs,passing_epa,passing_cpoe,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_first_downs,rushing_epa,fantasy_points_ppr
4791,00-0034796,Lamar Jackson,QB,2024,BAL,17,316,474,4172,41,4,23,2,198,172.278612,4.551942,1.015332,139,915,4,6,53,11.652933,430.38
1837,00-0033873,Patrick Mahomes,QB,2022,KC,17,435,648,5250,41,12,26,0,273,193.131205,3.572168,1.113468,61,358,4,4,28,24.309493,417.40
4524,00-0033873,Patrick Mahomes,QB,2018,KC,16,383,580,5097,50,12,26,3,237,221.761404,4.472190,0.969933,60,272,2,2,22,15.273858,417.08
5698,00-0034796,Lamar Jackson,QB,2019,BAL,15,265,401,3127,36,6,23,0,161,144.002652,3.812660,0.882087,176,1206,7,7,76,54.889087,415.68
3195,00-0034857,Josh Allen,QB,2021,BUF,17,409,646,4407,36,15,26,3,240,79.992858,2.097556,0.832295,122,763,6,5,56,55.646718,402.58


In [52]:
# List your counting stats to convert to per-game
qb_count_stats = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions',
               'sacks_suffered', 'sack_fumbles', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'pacr',
               'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs', 'rushing_epa',
               'fantasy_points_ppr']

qb_per_game = qb_filtered.copy()

for stat in qb_count_stats:
    qb_per_game[stat + '_per_game'] = (qb_per_game[stat] / qb_per_game['games'].replace(0, pd.NA)).round(1)

# Now you can drop or keep original total columns depending on your preference
# For example, keep per-game only:
qb_per_game = qb_per_game[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games'] +
                          [col for col in qb_per_game.columns if col.endswith('_per_game')]]

qb_per_game_filtered = qb_per_game[qb_per_game['games'] >= 4]

# Sort by player and season
qb_per_game_filtered = qb_per_game_filtered.sort_values(by = ['player_id', 'season'])

# Create target variable
qb_per_game_filtered['fantasy_points_ppr_per_game_next_year'] = (
    qb_per_game_filtered.groupby('player_id')['fantasy_points_ppr_per_game'].shift(-1)
)

qb_per_game_filtered.sort_values(by = 'fantasy_points_ppr_per_game', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,completions_per_game,attempts_per_game,passing_yards_per_game,passing_tds_per_game,passing_interceptions_per_game,sacks_suffered_per_game,sack_fumbles_per_game,passing_first_downs_per_game,passing_epa_per_game,passing_cpoe_per_game,pacr_per_game,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,rushing_epa_per_game,fantasy_points_ppr_per_game,fantasy_points_ppr_per_game_next_year
5698,00-0034796,Lamar Jackson,QB,2019,BAL,15,17.7,26.7,208.5,2.4,0.4,1.5,0.0,10.7,9.6,0.3,0.1,11.7,80.4,0.5,0.5,5.1,3.7,27.7,22.2
3684,00-0033077,Dak Prescott,QB,2020,DAL,5,30.2,44.4,371.2,1.8,0.8,2.0,0.6,18.6,6.4,0.1,0.2,3.6,18.6,0.6,0.0,1.6,2.0,27.1,20.0
4524,00-0033873,Patrick Mahomes,QB,2018,KC,16,23.9,36.2,318.6,3.1,0.8,1.6,0.2,14.8,13.9,0.3,0.1,3.8,17.0,0.1,0.1,1.4,1.0,26.1,20.5
4791,00-0034796,Lamar Jackson,QB,2024,BAL,17,18.6,27.9,245.4,2.4,0.2,1.4,0.1,11.6,10.1,0.3,0.1,8.2,53.8,0.2,0.4,3.1,0.7,25.3,NaN
2074,00-0036389,Jalen Hurts,QB,2022,PHI,15,20.4,30.7,246.7,1.5,0.4,2.5,0.3,11.0,4.8,0.2,0.1,11.0,50.7,0.9,0.3,4.9,3.3,25.2,21.0


In [53]:
qb_per_game_filtered_no_target = qb_per_game_filtered.drop(columns = 'fantasy_points_ppr_per_game_next_year')

print(f'There are {qb_per_game_filtered_no_target.isna().any(axis = 1).sum()} unexpected rows with NAN values\n')

qb_per_game_filtered_no_target[qb_per_game_filtered_no_target.isna().any(axis = 1)]

There are 6 unexpected rows with NAN values



,player_id,player_display_name,position,season,recent_team,games,completions_per_game,attempts_per_game,passing_yards_per_game,passing_tds_per_game,passing_interceptions_per_game,sacks_suffered_per_game,sack_fumbles_per_game,passing_first_downs_per_game,passing_epa_per_game,passing_cpoe_per_game,pacr_per_game,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,rushing_epa_per_game,fantasy_points_ppr_per_game
1269,00-0027796,Joe Webb,QB,2015,CAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.1,-0.1,0.0,0.0,0.0,-0.1,-0.0
659,00-0027796,Joe Webb,QB,2016,CAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.1
4139,00-0027796,Joe Webb,QB,2018,HOU,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.8
3798,00-0034177,Tim Boyle,QB,2020,GB,8,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,-0.3,NaN,NaN,1.6,-1.1,0.0,0.0,0.0,-0.8,-0.1
3154,00-0034438,Logan Woodside,QB,2021,TEN,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.5,-1.5,0.0,0.0,0.0,-1.1,-0.2
2447,00-0034577,Kyle Allen,QB,2023,BUF,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2.2,-2.2,0.0,0.0,0.0,-1.4,-0.2


In [54]:
qb_per_game_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 537 entries, 1109 to 5240
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player_id                              537 non-null    object 
 1   player_display_name                    537 non-null    object 
 2   position                               537 non-null    object 
 3   season                                 537 non-null    int64  
 4   recent_team                            537 non-null    object 
 5   games                                  537 non-null    int64  
 6   completions_per_game                   537 non-null    float64
 7   attempts_per_game                      537 non-null    float64
 8   passing_yards_per_game                 537 non-null    float64
 9   passing_tds_per_game                   537 non-null    float64
 10  passing_interceptions_per_game         537 non-null    float64
 11  sacks_s

In [55]:
qb_per_game_filtered = qb_per_game_filtered.dropna(subset = ['passing_epa_per_game', 'passing_cpoe_per_game',
                                                             'pacr_per_game', 'rushing_epa_per_game'])

qb_per_game_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531 entries, 1109 to 5240
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player_id                              531 non-null    object 
 1   player_display_name                    531 non-null    object 
 2   position                               531 non-null    object 
 3   season                                 531 non-null    int64  
 4   recent_team                            531 non-null    object 
 5   games                                  531 non-null    int64  
 6   completions_per_game                   531 non-null    float64
 7   attempts_per_game                      531 non-null    float64
 8   passing_yards_per_game                 531 non-null    float64
 9   passing_tds_per_game                   531 non-null    float64
 10  passing_interceptions_per_game         531 non-null    float64
 11  sacks_s

In [56]:
rb_all = all_seasons[all_seasons['position'] == 'RB']

rb_filtered = rb_all[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 
                      'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs', 'rushing_epa',
                      'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards',
                      'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'racr', 'target_share', 'air_yards_share',
                      'wopr', 'fantasy_points_ppr'
             ]].copy()

rb_filtered.sort_values(by = 'fantasy_points_ppr', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_first_downs,rushing_epa,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,racr,target_share,air_yards_share,wopr,fantasy_points_ppr
5529,00-0033280,Christian McCaffrey,RB,2019,CAR,16,287,1387,15,1,58,-14.251966,116,142,1005,4,0,111,987,58,50.483007,9.054054,0.236667,0.020947,0.369663,471.2
993,00-0032187,David Johnson,RB,2016,ARI,16,293,1239,16,5,76,-18.826080,80,120,879,4,0,529,661,42,23.366134,1.661626,0.188679,0.085268,0.342706,407.8
2370,00-0033280,Christian McCaffrey,RB,2023,SF,16,272,1459,14,2,86,14.388324,67,83,564,7,1,153,461,31,6.454787,3.686275,0.176221,0.038079,0.290986,391.3
4648,00-0034844,Saquon Barkley,RB,2018,NYG,16,261,1307,11,0,52,-7.855426,91,121,721,4,0,18,768,30,2.566300,40.055556,0.210801,0.004116,0.319083,385.8
4455,00-0033280,Christian McCaffrey,RB,2018,CAR,16,219,1098,7,2,56,7.454052,107,124,867,6,2,84,859,42,33.349967,10.321429,0.227941,0.020187,0.356043,385.5


In [57]:
rb_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1504 entries, 13 to 5810
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    1504 non-null   object 
 1   player_display_name          1504 non-null   object 
 2   position                     1504 non-null   object 
 3   season                       1504 non-null   int64  
 4   recent_team                  1504 non-null   object 
 5   games                        1504 non-null   int64  
 6   carries                      1504 non-null   int64  
 7   rushing_yards                1504 non-null   int64  
 8   rushing_tds                  1504 non-null   int64  
 9   rushing_fumbles              1504 non-null   int64  
 10  rushing_first_downs          1504 non-null   int64  
 11  rushing_epa                  1406 non-null   float64
 12  receptions                   1504 non-null   int64  
 13  targets               

In [58]:
rb_count_stats = ['carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs', 'rushing_epa', 'receptions', 
                  'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards', 
                  'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'fantasy_points_ppr']


rb_per_game = rb_filtered.copy()

for stat in rb_count_stats:
    rb_per_game[stat + '_per_game'] = (rb_per_game[stat] / rb_per_game['games'].replace(0, pd.NA))

# Now you can drop or keep original total columns depending on your preference
# For example, keep per-game only:
rb_per_game = rb_per_game[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 'racr', 'target_share', 
                           'air_yards_share', 'wopr'] + [col for col in rb_per_game.columns if col.endswith('_per_game')]]

rushing_cols = ['carries_per_game', 'rushing_yards_per_game', 'rushing_tds_per_game',
                'rushing_fumbles_per_game', 'rushing_first_downs_per_game', 'rushing_epa_per_game']

receiving_cols = ['receptions_per_game', 'targets_per_game', 'receiving_yards_per_game',
                  'receiving_tds_per_game', 'receiving_fumbles_per_game', 'receiving_air_yards_per_game',
                  'receiving_yards_after_catch_per_game', 'receiving_first_downs_per_game',
                  'receiving_epa_per_game', 'racr', 'target_share', 'air_yards_share', 'wopr']

fantasy_cols = ['fantasy_points_ppr_per_game']

# Build final ordered list
final_cols = ['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games']\
              + rushing_cols + receiving_cols + fantasy_cols

rb_per_game = rb_per_game[final_cols]

rb_per_game_filtered = rb_per_game[rb_per_game['games'] >= 4].round(2)

# Sort by player and season
rb_per_game_filtered = rb_per_game_filtered.sort_values(by = ['player_id', 'season'])

# Create target variable
rb_per_game_filtered['fantasy_points_ppr_per_game_next_year'] = (
    rb_per_game_filtered.groupby('player_id')['fantasy_points_ppr_per_game'].shift(-1)
)

#rb_per_game_filtered.sort_values(by = 'fantasy_points_ppr_per_game', ascending = False).head()
rb_per_game_filtered.head()

,player_id,player_display_name,position,season,recent_team,games,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,rushing_epa_per_game,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,racr,target_share,air_yards_share,wopr,fantasy_points_ppr_per_game,fantasy_points_ppr_per_game_next_year
1141,00-0023500,Frank Gore,RB,2015,IND,16,16.25,60.44,0.38,0.25,3.06,-3.11,2.12,3.62,16.69,0.06,0.0,-0.81,17.06,0.62,-0.03,-20.54,0.10,-0.00,0.14,12.09,13.39
571,00-0023500,Frank Gore,RB,2016,IND,16,16.44,64.06,0.25,0.12,3.12,-1.53,2.38,2.94,17.31,0.25,0.0,0.06,17.88,0.88,0.63,277.00,0.08,0.00,0.12,13.39,10.85
13,00-0023500,Frank Gore,RB,2017,IND,16,16.31,60.06,0.19,0.19,3.12,-1.50,1.81,2.38,15.31,0.06,0.0,-1.62,17.19,0.56,0.30,-9.42,0.08,-0.01,0.12,10.85,7.33
4092,00-0023500,Frank Gore,RB,2018,MIA,14,11.14,51.57,0.00,0.07,1.79,-0.28,0.86,1.14,8.86,0.07,0.0,1.50,8.21,0.57,0.49,5.90,0.04,0.01,0.06,7.33,6.33
5255,00-0023500,Frank Gore,RB,2019,BUF,15,11.07,39.93,0.13,0.00,2.20,-2.19,0.87,1.07,6.67,0.00,0.0,0.93,5.87,0.33,0.07,7.14,0.03,0.00,0.05,6.33,6.68


In [59]:
rb_per_game_filtered_no_target = rb_per_game_filtered.drop(columns = 'fantasy_points_ppr_per_game_next_year')

print(f'There are {rb_per_game_filtered_no_target.isna().any(axis = 1).sum()} unexpected rows with NAN values\n')

rb_per_game_filtered_no_target[rb_per_game_filtered_no_target.isna().any(axis = 1)]

There are 74 unexpected rows with NAN values



,player_id,player_display_name,position,season,recent_team,games,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,rushing_epa_per_game,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,racr,target_share,air_yards_share,wopr,fantasy_points_ppr_per_game
1218,00-0026932,Cedric Peerman,RB,2015,CIN,12,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00
625,00-0026932,Cedric Peerman,RB,2016,CIN,4,1.50,3.75,0.00,0.00,0.00,-0.61,0.00,0.25,0.00,0.00,0.0,0.00,0.00,0.00,-0.12,NaN,0.00,0.00,0.00,0.38
99,00-0028063,Taiwan Jones,RB,2017,BUF,5,0.00,0.00,0.00,0.00,0.00,NaN,0.20,0.40,2.20,0.00,0.0,4.80,2.00,0.20,0.08,0.46,0.00,0.01,0.01,0.42
4156,00-0028063,Taiwan Jones,RB,2018,BUF,5,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.40,0.00,0.00,0.0,6.00,0.00,0.00,-0.37,0.00,0.00,0.01,0.01,0.00
3526,00-0028063,Taiwan Jones,RB,2020,BUF,8,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.25,0.00,0.00,0.0,5.50,0.00,0.00,-0.13,0.00,0.00,0.01,0.01,0.00
2881,00-0028063,Taiwan Jones,RB,2021,BUF,7,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00
1686,00-0028063,Taiwan Jones,RB,2022,BUF,7,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00
112,00-0028121,Jordan Todman,RB,2017,HOU,9,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00
711,00-0028581,Patrick DiMarco,RB,2016,ATL,10,0.00,0.00,0.00,0.00,0.00,NaN,0.70,1.00,5.20,0.10,0.0,2.20,2.90,0.30,-0.05,2.36,0.02,0.00,0.03,1.82
1332,00-0028795,Chase Reynolds,RB,2015,LA,8,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00


In [60]:
rb_per_game_filtered = rb_per_game_filtered.dropna(subset = ['rushing_epa_per_game', 'receiving_epa_per_game', 'racr'])

rb_per_game_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1162 entries, 1141 to 5243
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player_id                              1162 non-null   object 
 1   player_display_name                    1162 non-null   object 
 2   position                               1162 non-null   object 
 3   season                                 1162 non-null   int64  
 4   recent_team                            1162 non-null   object 
 5   games                                  1162 non-null   int64  
 6   carries_per_game                       1162 non-null   float64
 7   rushing_yards_per_game                 1162 non-null   float64
 8   rushing_tds_per_game                   1162 non-null   float64
 9   rushing_fumbles_per_game               1162 non-null   float64
 10  rushing_first_downs_per_game           1162 non-null   float64
 11  rushin

In [61]:
wr_all = all_seasons[all_seasons['position'] == 'WR']

wr_filtered = wr_all[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 
                      'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs',
                      'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards',
                      'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'racr', 'target_share', 'air_yards_share',
                      'wopr', 'fantasy_points_ppr'
             ]].copy()

wr_filtered.sort_values(by = 'fantasy_points_ppr', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_first_downs,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,racr,target_share,air_yards_share,wopr,fantasy_points_ppr
3098,00-0033908,Cooper Kupp,WR,2021,LA,17,4,18,0,0,1,145,191,1947,16,0,1641,846,89,112.935848,1.186472,0.317276,0.319634,0.699657,439.5
2584,00-0036358,CeeDee Lamb,WR,2023,DAL,17,14,113,2,0,6,135,181,1749,12,3,1722,680,81,103.519985,1.015679,0.299174,0.359199,0.700200,403.2
4942,00-0036900,Ja'Marr Chase,WR,2024,CIN,17,3,32,0,0,2,127,175,1708,17,0,1526,787,75,77.010441,1.119266,0.278662,0.330733,0.649506,403.0
1268,00-0027793,Antonio Brown,WR,2015,PIT,16,3,28,0,0,1,136,193,1834,10,1,2110,593,84,58.203196,0.869194,0.330479,0.358234,0.746483,390.2
2357,00-0033040,Tyreek Hill,WR,2023,MIA,16,6,15,0,0,2,119,171,1799,13,1,1847,652,83,91.748831,0.974012,0.311475,0.428936,0.767469,376.4


In [62]:
wr_count_stats = ['carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_first_downs', 'receptions', 
                  'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards', 
                  'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'fantasy_points_ppr']


wr_per_game = wr_filtered.copy()

for stat in wr_count_stats:
    wr_per_game[stat + '_per_game'] = (wr_per_game[stat] / wr_per_game['games'].replace(0, pd.NA))

# Now you can drop or keep original total columns depending on your preference
# For example, keep per-game only:
wr_per_game = wr_per_game[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 'racr', 'target_share', 
                           'air_yards_share', 'wopr'] + [col for col in wr_per_game.columns if col.endswith('_per_game')]]

rushing_cols = ['carries_per_game', 'rushing_yards_per_game', 'rushing_tds_per_game',
                'rushing_fumbles_per_game', 'rushing_first_downs_per_game']

receiving_cols = ['receptions_per_game', 'targets_per_game', 'receiving_yards_per_game',
                  'receiving_tds_per_game', 'receiving_fumbles_per_game', 'receiving_air_yards_per_game',
                  'receiving_yards_after_catch_per_game', 'receiving_first_downs_per_game',
                  'receiving_epa_per_game', 'racr', 'target_share', 'air_yards_share', 'wopr']

fantasy_cols = ['fantasy_points_ppr_per_game']

# Build final ordered list
final_cols = ['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games']\
              + receiving_cols + rushing_cols + fantasy_cols

wr_per_game = wr_per_game[final_cols]

wr_per_game_filtered = wr_per_game[wr_per_game['games'] >= 4].round(2)

# Sort by player and season
wr_per_game_filtered = wr_per_game_filtered.sort_values(by = ['player_id', 'season'])

# Create target variable
wr_per_game_filtered['fantasy_points_ppr_per_game_next_year'] = (
    wr_per_game_filtered.groupby('player_id')['fantasy_points_ppr_per_game'].shift(-1)
)

wr_per_game_filtered.sort_values(by = 'fantasy_points_ppr_per_game', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,racr,target_share,air_yards_share,wopr,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,fantasy_points_ppr_per_game,fantasy_points_ppr_per_game_next_year
3098,00-0033908,Cooper Kupp,WR,2021,LA,17,8.53,11.24,114.53,0.94,0.00,96.53,49.76,5.24,6.64,1.19,0.32,0.32,0.70,0.24,1.06,0.00,0.0,0.06,25.85,22.38
3599,00-0031381,Davante Adams,WR,2020,GB,14,8.21,10.64,98.14,1.29,0.07,95.00,42.64,5.21,5.53,1.03,0.30,0.32,0.67,0.00,0.00,0.00,0.0,0.00,25.60,21.52
1268,00-0027793,Antonio Brown,WR,2015,PIT,16,8.50,12.06,114.62,0.62,0.06,131.88,37.06,5.25,3.64,0.87,0.33,0.36,0.75,0.19,1.75,0.00,0.0,0.06,24.39,20.49
2584,00-0036358,CeeDee Lamb,WR,2023,DAL,17,7.94,10.65,102.88,0.71,0.18,101.29,40.00,4.76,6.09,1.02,0.30,0.36,0.70,0.82,6.65,0.12,0.0,0.35,23.72,17.56
4942,00-0036900,Ja'Marr Chase,WR,2024,CIN,17,7.47,10.29,100.47,1.00,0.00,89.76,46.29,4.41,4.53,1.12,0.28,0.33,0.65,0.18,1.88,0.00,0.0,0.12,23.71,NaN


In [63]:
wr_per_game_filtered_no_target = wr_per_game_filtered.drop(columns = 'fantasy_points_ppr_per_game_next_year')

print(f'There are {wr_per_game_filtered_no_target.isna().any(axis = 1).sum()} unexpected rows with NAN values\n')

wr_per_game_filtered_no_target[wr_per_game_filtered_no_target.isna().any(axis = 1)]

There are 49 unexpected rows with NAN values



,player_id,player_display_name,position,season,recent_team,games,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,racr,target_share,air_yards_share,wopr,carries_per_game,rushing_yards_per_game,rushing_tds_per_game,rushing_fumbles_per_game,rushing_first_downs_per_game,fantasy_points_ppr_per_game
1159,00-0024272,Devin Hester,WR,2015,ATL,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
583,00-0024272,Devin Hester,WR,2016,BAL,12,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
587,00-0024535,Eric Weems,WR,2016,ATL,16,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
1176,00-0025460,Jacoby Jones,WR,2015,PIT,9,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
47,00-0026293,Matthew Slater,WR,2017,NE,6,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
4113,00-0026293,Matthew Slater,WR,2018,NE,9,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
5270,00-0026293,Matthew Slater,WR,2019,NE,7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.86
3501,00-0026293,Matthew Slater,WR,2020,NE,6,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
2860,00-0026293,Matthew Slater,WR,2021,NE,10,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
1672,00-0026293,Matthew Slater,WR,2022,NE,8,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00


In [64]:
wr_per_game_filtered = wr_per_game_filtered.dropna(subset = ['receiving_epa_per_game', 'racr'])

wr_per_game_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1863 entries, 1113 to 5242
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player_id                              1863 non-null   object 
 1   player_display_name                    1863 non-null   object 
 2   position                               1863 non-null   object 
 3   season                                 1863 non-null   int64  
 4   recent_team                            1863 non-null   object 
 5   games                                  1863 non-null   int64  
 6   receptions_per_game                    1863 non-null   float64
 7   targets_per_game                       1863 non-null   float64
 8   receiving_yards_per_game               1863 non-null   float64
 9   receiving_tds_per_game                 1863 non-null   float64
 10  receiving_fumbles_per_game             1863 non-null   float64
 11  receiv

In [65]:
te_all = all_seasons[all_seasons['position'] == 'TE']

te_filtered = te_all[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 
                      'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards',
                      'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'racr', 'target_share', 'air_yards_share',
                      'wopr', 'fantasy_points_ppr'
             ]].copy()

te_filtered.sort_values(by = 'fantasy_points_ppr', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,racr,target_share,air_yards_share,wopr,fantasy_points_ppr
1706,00-0030506,Travis Kelce,TE,2022,KC,17,110,152,1338,12,1,1074,657,78,66.036887,1.245810,0.247557,0.227929,0.530886,316.30
3569,00-0030506,Travis Kelce,TE,2020,KC,15,105,145,1416,11,1,1243,587,79,75.219912,1.139179,0.237316,0.243582,0.526481,312.76
3173,00-0034753,Mark Andrews,TE,2021,BAL,17,107,153,1361,9,1,1574,450,75,56.071777,0.864676,0.258883,0.300841,0.598914,301.10
4241,00-0030506,Travis Kelce,TE,2018,KC,16,103,150,1336,10,2,1375,568,68,69.233599,0.971636,0.265957,0.260762,0.581470,294.60
4217,00-0030061,Zach Ertz,TE,2018,PHI,16,116,156,1163,8,1,1130,353,66,19.195038,1.029204,0.263514,0.253534,0.572744,280.30


In [66]:
te_count_stats = ['receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_air_yards', 
                  'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'fantasy_points_ppr']


te_per_game = te_filtered.copy()

for stat in te_count_stats:
    te_per_game[stat + '_per_game'] = (te_per_game[stat] / te_per_game['games'].replace(0, pd.NA))

# Now you can drop or keep original total columns depending on your preference
# For example, keep per-game only:
te_per_game = te_per_game[['player_id', 'player_display_name', 'position', 'season', 'recent_team', 'games', 'racr', 'target_share', 
                           'air_yards_share', 'wopr'] + [col for col in te_per_game.columns if col.endswith('_per_game')]]

te_per_game_filtered = te_per_game[te_per_game['games'] >= 4].round(2)

# Sort by player and season
te_per_game_filtered = te_per_game_filtered.sort_values(by = ['player_id', 'season'])

# Create target variable
te_per_game_filtered['fantasy_points_ppr_per_game_next_year'] = (
    te_per_game_filtered.groupby('player_id')['fantasy_points_ppr_per_game'].shift(-1)
)

te_per_game_filtered.sort_values(by = 'fantasy_points_ppr_per_game', ascending = False).head()

,player_id,player_display_name,position,season,recent_team,games,racr,target_share,air_yards_share,wopr,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,fantasy_points_ppr_per_game,fantasy_points_ppr_per_game_next_year
3569,00-0030506,Travis Kelce,TE,2020,KC,15,1.14,0.24,0.24,0.53,7.00,9.67,94.40,0.73,0.07,82.87,39.13,5.27,5.01,20.85,16.42
1706,00-0030506,Travis Kelce,TE,2022,KC,17,1.25,0.25,0.23,0.53,6.47,8.94,78.71,0.71,0.06,63.18,38.65,4.59,3.88,18.61,14.63
4241,00-0030506,Travis Kelce,TE,2018,KC,16,0.97,0.27,0.26,0.58,6.44,9.38,83.50,0.62,0.12,85.94,35.50,4.25,4.33,18.41,15.89
3173,00-0034753,Mark Andrews,TE,2021,BAL,17,0.86,0.26,0.30,0.60,6.29,9.00,80.06,0.53,0.06,92.59,26.47,4.41,3.30,17.71,12.70
4217,00-0030061,Zach Ertz,TE,2018,PHI,16,1.03,0.26,0.25,0.57,7.25,9.75,72.69,0.50,0.06,70.62,22.06,4.12,1.20,17.52,14.37


In [67]:
te_per_game_filtered_no_target = te_per_game_filtered.drop(columns = 'fantasy_points_ppr_per_game_next_year')

print(f'There are {te_per_game_filtered_no_target.isna().any(axis = 1).sum()} unexpected rows with NAN values\n')

te_per_game_filtered_no_target[te_per_game_filtered_no_target.isna().any(axis = 1)]

There are 8 unexpected rows with NAN values



,player_id,player_display_name,position,season,recent_team,games,racr,target_share,air_yards_share,wopr,receptions_per_game,targets_per_game,receiving_yards_per_game,receiving_tds_per_game,receiving_fumbles_per_game,receiving_air_yards_per_game,receiving_yards_after_catch_per_game,receiving_first_downs_per_game,receiving_epa_per_game,fantasy_points_ppr_per_game
725,00-0029091,Bradley Sowell,TE,2016,SEA,4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
130,00-0029091,Bradley Sowell,TE,2017,CHI,4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
994,00-0032188,Vince Mayle,TE,2016,DAL,5,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
398,00-0032661,Ben Braunecker,TE,2017,CHI,5,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
3014,00-0032986,J.P. Holtz,TE,2021,CHI,5,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
1868,00-0034193,Kevin Rader,TE,2022,TEN,6,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00
3433,00-0036825,Feleipe Franks,TE,2021,ATL,4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.35
5201,00-0039686,Colson Yankoff,TE,2024,WAS,4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00


In [68]:
te_per_game_filtered = te_per_game_filtered.dropna(subset = ['receiving_epa_per_game', 'racr'])

te_per_game_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1020 entries, 1118 to 5236
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player_id                              1020 non-null   object 
 1   player_display_name                    1020 non-null   object 
 2   position                               1020 non-null   object 
 3   season                                 1020 non-null   int64  
 4   recent_team                            1020 non-null   object 
 5   games                                  1020 non-null   int64  
 6   racr                                   1020 non-null   float64
 7   target_share                           1020 non-null   float64
 8   air_yards_share                        1020 non-null   float64
 9   wopr                                   1020 non-null   float64
 10  receptions_per_game                    1020 non-null   float64
 11  target

In [69]:
qb_per_game_filtered.to_csv('../data/processed/qb_per_game_filtered.csv', index = False)
rb_per_game_filtered.to_csv('../data/processed/rb_per_game_filtered.csv', index = False)
wr_per_game_filtered.to_csv('../data/processed/wr_per_game_filtered.csv', index = False)
te_per_game_filtered.to_csv('../data/processed/te_per_game_filtered.csv', index = False)